# Tract-Patch Search

In [ ]:
# Client for Dask distributed computing
from dask.distributed import Client

# For reading the HATS catalogs and performing the cross-match
import lsdb

# For reading and working with skymaps for tract and patch searches
import lsst.skymap
import pickle

# For accessing Rubin-specific lsdb methods; in this case, tract_patch_search
from lsdb_rubin import tract_patch_search
lsdb.catalog.Catalog.tract_patch_search = tract_patch_search

In [ ]:
client = Client(n_workers=4, memory_limit="auto")
client

## Load GAIA and the LSST skymap

In [ ]:
# Load GAIA DR3 data.

gaia = lsdb.read_hats('https://data.lsdb.io/hats/gaia_dr3/gaia', margin_cache='https://data.lsdb.io/hats/gaia_dr3/gaia_10arcs')
gaia

In [ ]:
# Load the LSST skymap.

lsst_skymap_path = "/sdf/home/o/olynn/LINCC/Skymaps/skyMap_lsst_cells_v1_skymaps.pickle"

with open(lsst_skymap_path, "rb") as f:
    lsst_skymap = pickle.load(f)
    print(lsst_skymap)

## Use RA/Dec to get tract and patch IDs

In [ ]:
# Get tract and patch numbers for a given RA and Dec.

ra_float = 42.0
dec_float = 2.0

longitude = lsst.geom.Angle(ra_float, lsst.geom.degrees)
latitude = lsst.geom.Angle(dec_float, lsst.geom.degrees)
sphere_point = lsst.geom.SpherePoint(longitude, latitude)

tract_patch_list = lsst_skymap.findTractPatchList([sphere_point])
tract_patch_list

tract_index = tract_patch_list[0][0]._id
patch_index = tract_patch_list[0][1][0]._sequentialIndex

print(f"Tract: {tract_index}, Patch: {patch_index}")

In [ ]:
# Get tract and patch numbers for a given RA and Dec.

ra_float = 300
dec_float = -15

longitude = lsst.geom.Angle(ra_float, lsst.geom.degrees)
latitude = lsst.geom.Angle(dec_float, lsst.geom.degrees)
sphere_point = lsst.geom.SpherePoint(longitude, latitude)

tract_patch_list = lsst_skymap.findTractPatchList([sphere_point])
tract_patch_list

tract_index = tract_patch_list[0][0]._id
patch_index = tract_patch_list[0][1][0]._sequentialIndex

print(f"Tract: {tract_index}, Patch: {patch_index}")

# TODO : Account for multiple tracts/patches (eg, ra: 42, dec: 3)

## Search by tract and patch IDs

### Search by tract

In [ ]:
# Only specify the tract (and not patch) to search by tract.

gaia_tract = gaia.tract_patch_search(
    skymap=lsst_skymap,
    tract=tract_index,
    fine=True,
)
gaia_tract

In [ ]:
gaia_tract.plot_pixels(
    plot_title="Gaia DR3 Tract Search",
    fc="#00000000",
    ec="red",
    alpha=0.5,
)

### Search by tract and patch

In [ ]:
# Specify both tract and patch to search by tract and patch.

gaia_tract_patch = gaia.tract_patch_search(
    skymap=lsst_skymap,
    tract=tract_index,
    patch=patch_index,
    fine=True,
)
gaia_tract_patch

In [ ]:
gaia_tract_patch.plot_pixels(
    plot_title="Gaia DR3 Tract and Patch Search",
    fc="#00000000",
    ec="red",
    alpha=0.5,
)

In [ ]:
df = gaia_tract_patch.compute()
df

In [ ]:
from lsdb.core.plotting import plot_points

plot_points.plot_points(
    df,
    ra_column="ra",
    dec_column="dec",
    title="Gaia DR3 Tract and Patch Search",
)

### Set `user_inner` to True to search the "inner" region of the tract or patch

Read more at the [LSST Skymap Docs](https://github.com/lsst/skymap/blob/main/doc/main.dox):

> Tracts contain an inner region described by a collection of vertices. The inner regions exactly tile the portion of sky covered by the sky map. All pixels beyond the inner region provide overlap with neighboring tracts.

> Patches contain rectangular inner and outer regions. The inner regions exactly tile the tract, and all patches in a tract have the same inner dimensions. Each patch has a border around the inner region to provide some overlap with adjacent patches, but there is no border on patch edges that lie against tract boundaries.

Note that `use_inner` may be specified for either a tract or a patch search.

In [ ]:
gaia_tract_patch_outer = gaia.tract_patch_search(
    skymap=lsst_skymap,
    tract=tract_index,
    patch=patch_index,
    fine=True,
    use_inner=True,
)
gaia_tract_patch_outer

In [ ]:
df = gaia_tract_patch_outer.compute()
df